<a href="https://colab.research.google.com/github/DEP04929/ESICMDatathon2026/blob/main/WeanCombineRaw_20260117.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites for Amsterdam UMC DB


In [ ]:
# sets *your* project id
PROJECT_ID = "esicmdatathon2026" #@param {type:"string"}


In [ ]:
# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [ ]:
import os
from google.colab import auth

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

auth.authenticate_user()
print('Authenticated')


In [ ]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 30000


In [ ]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# Vent data detect weaning
Only very simplified checks worked. I checked for PEEP, FiO2 and TV for existence of ventilation and counted hours. Failure is if any of these parameters return charting in 48 hours, or death within 48 hours.

In [ ]:
%%bigquery ventpat --project $PROJECT_ID
with vent as
(
  with vt as ( select person_id, measurement_datetime
  , case when measurement_concept_id = 3022875 then 'PEEP'
       when measurement_concept_id in (3025408, 2000000204) then 'FiO2'
       when measurement_concept_id = 3012410 then 'TV'
       -- when measurement_concept_id = 2000000250 then 'CPAP'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
        3022875 -- Positive end expiratory pressure setting Ventilator
        , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
        , 2000000204 -- fio2 setting
        , 3012410 -- Tidal volume setting Ventilator
       -- , 2000000250 -- Ventilation mode Ventilator non-invasive, very few entries, not used!
  )
  -- and person_id = 18 -- 407 -- 18 -- testing only
  and value_as_number is not null
  )
  select * from vt
  PIVOT ( max(value_as_number) for concept_name in ('PEEP', 'FiO2', 'TV'))
  order by person_id, measurement_datetime
),
hr as (
select person_id, measurement_datetime
, value_as_number as hf_ekg
from measurement m
where measurement_concept_id in (
 	21490872 --Heart rate.beat-to-beat by EKG
)
and person_id in (select distinct person_id from vent ) -- i only am interested in vent patients
),
tube as (
  select person_id, measurement_datetime, value_as_number as tubesize
  from measurement m
  where measurement_concept_id in ( 36305611 -- Tracheostomy tube diameter
                                  , 21491186 -- Endotracheal tube Diameter
  )
  order by person_id, measurement_datetime
), died as (
select distinct person_id, death_datetime
  from death
  where person_id in (select person_id from vent)
)
, ventall as (
-- make a summary of all values before checking wean episodes
  select hr.person_id, hr.measurement_datetime, hr.hf_ekg
  , vent.PEEP, vent.FiO2, vent.TV
  , tube.tubesize, died.death_datetime
  , case when date_diff(died.death_datetime, vent.measurement_datetime, DAY) <=2
        then 1 else 0 end as death_in_48hrs
  , (select count(*)
          from vent b
          where vent.person_id = b.person_id
          and b.measurement_datetime between timestamp_add( vent.measurement_datetime, interval 1 MINUTE)
                                  and timestamp_add(vent.measurement_datetime, interval 2 HOUR )
          and (b.FiO2 >0 or b.PEEP >0 or b.TV > 0) -- any vent?
                                  ) as check2hrs
  , (select count(*)
          from vent b
          where vent.person_id = b.person_id
          and b.measurement_datetime between timestamp_add( vent.measurement_datetime, interval 1 MINUTE)
                                  and timestamp_add(vent.measurement_datetime, interval 6 HOUR )
          and (b.FiO2 >0 or b.PEEP >0 or b.TV > 0) -- any vent?
                                  ) as check6hrs
  -- problems with very short stays < 48 hours
  , (select count(*)
          from vent b
          where vent.person_id = b.person_id
          and b.measurement_datetime between timestamp_add( vent.measurement_datetime, interval 1 MINUTE)
                                  and timestamp_add(vent.measurement_datetime, interval 48 HOUR )
          and (b.FiO2 >0 or b.PEEP >0 or b.TV > 0 -- resume vent?
                or tube.tubesize >0 ) -- tube in 48hrs?
                                  ) as check48hrs
  from hr
  left outer join vent
  on vent.person_id = hr.person_id
  and vent.measurement_datetime = hr.measurement_datetime
  left outer join tube
  on hr.person_id = tube.person_id
  and hr.measurement_datetime between timestamp_add(tube.measurement_datetime , INTERVAL -30 MINUTE)
              and TIMESTAMP_ADD(tube.measurement_datetime , INTERVAL 30 MINUTE)
  left outer join died
  on hr.person_id = died.person_id
),
wean_list as (
select a.person_id, a.measurement_datetime as wean_start
, a.hf_ekg
, case when check48hrs > 1 or death_in_48hrs = 1
       then 0 else 1 end as outcome -- failed = 0, success = 1
from ventall a
where check2hrs = 1 and check6hrs = 1  --this means weaning for 2-6 hrs.
),
hf_6hrsago as (
  with hf as (
  select m.person_id, w.wean_start, measurement_datetime
  , case when measurement_concept_id = 21490872 then 'hf_ekg'
      --  when measurement_concept_id = 40762499 then 'SpO2'
        when measurement_concept_id = 3020891 then 'temp'
        when measurement_concept_id = 3007194 then 'GCS'
        when measurement_concept_id = 2000000016 then 'RASS'
        when measurement_concept_id = 21490712 then 'LVCI'
        when measurement_concept_id = 3005555 then 'LVCO'
        end as measurement_type
  , value_as_number
  from measurement m inner join wean_list w on w.person_id = m.person_id
  /* replace Interval here if you want a different time before estimated wean start */
  and m.measurement_datetime between timestamp_add(w.wean_start, INTERVAL -6 HOUR)
                                  and w.wean_start
  where measurement_concept_id in (
    21490872 --Heart rate.beat-to-beat by EKG
    -- , 40762499 -- Oxygen saturation in Arterial blood by Pulse oximetry
    , 3020891 -- Body temperature
    , 3007194  -- Glasgow coma score total
    , 2000000016 -- Richmond agitation-sedation scale score
    , 21490712 -- Left ventricular Cardiac index
    , 3005555 -- Left ventricular Cardiac output
    )
    and value_as_number < 300 -- remove implausible heart rates
  )
  select *
  from hf
  PIVOT ( max(value_as_number) for measurement_type in ('hf_ekg'--, 'SpO2'
                      , 'temp', 'GCS', 'RASS', 'LVCI', 'LVCO'))
)
select w.person_id, w.wean_start, w.outcome, h.measurement_datetime, h.hf_ekg, h.temp, h.GCS, h.RASS, h.LVCI, h.LVCO
 from
 wean_list w inner join hf_6hrsago h
  on w.person_id = h.person_id and w.wean_start = h.wean_start
order by person_id, measurement_datetime


# Check weaning list

In [ ]:
ventpat.describe()


In [ ]:
# ventpat[ventpat['RASS']>0]  # plausibilty check
ventpat[ventpat['person_id']==1470] # case random check

# Get the rest of the data

In [ ]:
# @title Arterial blood pressure
%%bigquery bp --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, bp as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 21490853 then 'ABPs'
     when measurement_concept_id = 21490851 then 'ABPd'
     when measurement_concept_id = 21490852 then 'ABPm'
     when measurement_concept_id = 3000333 then 'CVPm'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 21490851 -- ABP d
  , 21490852 -- ABP m
  , 21490853 -- ABPs
  , 3000333 -- CVP m
)
)
select * from bp
PIVOT ( max(value_as_number) for measure_type in ('ABPs', 'ABPd', 'ABPm', 'CVPm'))
order by person_id, measurement_datetime

In [ ]:
# merge bp values
cols_to_use = bp.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, bp[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
# clean up to save space
del bp

In [ ]:
# @title Breath rate or respiratory rate
%%bigquery rr --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
)
, rr as (
select person_id, measurement_datetime, value_as_number
, case when measurement_concept_id = 3043148 then 'BRvent'
     when measurement_concept_id = 3026892 then 'BRspontv'
     when measurement_concept_id = 1175625 then 'BRspont'
     when measurement_concept_id = 3024171 then 'RR'
     end as measure_type
from measurement m
where person_id in (select person_id from vlist)
  and measurement_concept_id in ( 3043148 --Breath rate mechanical
                                 , 3026892 -- Breath rate spontaneous --on ventilator
                                 , 1175625 -- Breath rate spontaneous
                                 , 3024171 -- Respiratory rate
                                 )
)
select * from rr
PIVOT ( max(value_as_number) for measure_type in ('BRvent', 'BRspontv','BRspont', 'RR'))
order by person_id, measurement_datetime

In [ ]:
# merge rr values
cols_to_use = rr.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, rr[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
# clean up to save space
del rr

In [ ]:
# @title BGA
%%bigquery bga --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
), br as ( select person_id, measurement_datetime
  , case when measurement_concept_id = 3010421 then 'pH'
     when measurement_concept_id in (3027315, 3027801) then 'PaO2'
		 when measurement_concept_id in (3027946, 3013290) then 'PaCO2'
		 when measurement_concept_id in (3006576, 3008152) then 'HCO3'
		 when measurement_concept_id in (3012501, 3003396) then 'BE'
     when measurement_concept_id in (3024928, 1616373) then 'SvO2'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
         3010421 -- pH of Blood
	      ,3027315 -- Oxygen [Partial pressure] in Blood
        ,3027801 -- Oxygen [Partial pressure] in Arterial blood
				,3013290 -- Carbon dioxide [Partial pressure] in Blood
        ,3027946 -- Carbon dioxide [Partial pressure] in Arterial blood
				,3006576 -- Bicarbonate [Moles/volume] in Blood
        ,3008152 -- Bicarbonate [Moles/volume] in Arterial blood
				,3012501 -- Base excess in Blood by calculation
        ,3003396 -- Base excess in Arterial blood by calculation
				,3024928 -- Oxygen saturation in Venous blood
        ,1616373 -- Oxygen saturation in Central venous blood
  )
  and value_as_number is not null
  and (measurement_concept_id not in (3027315, 3027801) or
        -- there are PaO2 entries that are converted to mmHg, but more values are in kPA
        -- ignoring those that are mmHg ?
       ( measurement_concept_id in (3027315, 3027801) and unit_source_value = 'kPa') )
  and person_id in (select person_id from vlist)
  )
  select * from br
  PIVOT (max(value_as_number) for concept_name in ('pH', 'PaO2', 'PaCO2', 'HCO3','BE','SvO2'))
  order by person_id, measurement_datetime

In [ ]:
# merge BGA values
cols_to_use = bga.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, bga[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
# clean up to save space
del bga

In [ ]:
# @title PEEP, TV, MV, Lung compliance, Resistance
%%bigquery peep --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
 vent as
( select person_id, measurement_datetime
  , case when measurement_concept_id = 3012410 then 'TV'
       when measurement_concept_id = 36303816 then 'TVinsp'
       when measurement_concept_id = 2000000222 then 'MVspont'
       when measurement_concept_id = 3045410 then 'MVset'
       when measurement_concept_id in (3022875, 3016226) then 'PEEP'
       when measurement_concept_id = 3035822 then 'IntrinsicPEEP'
       when measurement_concept_id = 2000000211 then 'IPEEP'
       when measurement_concept_id = 21490581 then 'COMPL'
       when measurement_concept_id = 21490740 then 'RESIS'
       end as concept_name
  , value_as_number
  from measurement m
  where measurement_concept_id in (
        3012410 -- Tidal volume setting Ventilator
        , 36303816 -- Tidal volume.inspired
        , 3022875 -- Positive end expiratory pressure setting Ventilator
        , 3016226 -- PEEP Respiratory system
        , 2000000222 -- Minute volume spontaneous
        , 3045410 -- Minute volume setting Ventilator
        , 3035822 -- Intrinsic PEEP Respiratory system
        , 2000000211 -- Inspiratory Pressure Above PEEP
        , 21490581 -- Lung compliance
        , 21490740 -- Airway resistance
  )
  and value_as_number is not null
  and person_id in (select person_id from vlist)
)
select * from vent
  PIVOT (max(value_as_number) for concept_name in ('MVspont','MVset', 'TV', 'TVinsp'
            ,'PEEP', 'IntrinsicPEEP', 'IPEEP', 'COMPL', 'RESIS'))
order by person_id, measurement_datetime

In [ ]:
# merge peep values
cols_to_use = peep.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, peep[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
del peep

In [ ]:
# @title FiO2
%%bigquery FiO2 --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
fio2 as (
select person_id, measurement_datetime
, case when measurement_concept_id in ( 3025408, 2000000204) then 'FiO2'
       when measurement_concept_id = 2000000203 then 'FiO2niv'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (2000000203 -- FiO2 NIV
  , 3025408 -- Oxygen/Inspired gas Respiratory system by O2 Analyzer --on ventilator
  --, 2000000204 -- fio2 setting
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from fio2
PIVOT (max(value_as_number) for concept_name in ('FiO2', 'FiO2niv'))
order by person_id, measurement_datetime

In [ ]:
# merge FiO2 values
cols_to_use = FiO2.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, FiO2[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
del FiO2

In [ ]:
# @title Labs: creatinine, bilirubin, hemoglobin and co.
%%bigquery lab --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
c as (
select person_id, measurement_datetime
, case when measurement_concept_id = 3020564 then 'Cr'
       when measurement_concept_id in (3043995, 3005772) then 'Br'
       when measurement_concept_id = 40762351 then 'Hb'
       when measurement_concept_id = 3007461 then 'Plt'
       when measurement_concept_id = 3010813 then 'wbc'
       when measurement_concept_id in (3000285, 3019550) then 'Na'
       When measurement_concept_id = 3020779 then 'Urea'
       when measurement_concept_id = 3024561 then 'Alb'
       when measurement_concept_id = 3009542 then 'Hct'
       when measurement_concept_id = 3005456 then 'K'
       when measurement_concept_id in (3047181, 3018405) then 'Lactate'
       end as concept_name
, value_as_number
from measurement m
where measurement_concept_id in (
   3020564 -- Creatinine [Moles/volume] in Serum or Plasma
  , 3005772 -- Bilirubin.conjugated [Moles/volume] in Serum or Plasma
  , 3043995 -- Bilirubin.conjugated+indirect [Moles/volume] in Serum or Plasma
  , 40762351 -- Hemoglobin [Moles/volume] in Blood
  , 3007461 -- Platelets [#/volume] in Blood
  , 3010813 -- Leukocytes [#/volume] in Blood
  , 3000285 -- Sodium [Moles/volume] in Blood
  , 3019550 -- Sodium [Moles/volume] in Serum or Plasma
  , 3020779 --Urea [Moles/volume] in Serum or Plasma
  , 3024561 -- Albumin [Mass/volume] in Serum or Plasma
  , 3009542 -- Hematocrit [Volume Fraction] of Blood by calculation
  , 3005456 -- Potassium [Moles/volume] in Blood
  , 3047181 -- Lactate [Moles/volume] in Blood
  , 3018405 -- Lactate [Moles/volume] in Arterial blood
)
  and person_id in (select person_id from vlist)
  and value_as_number > 0
)
select * from c
PIVOT (max(value_as_number) for concept_name in ('Cr', 'Br', 'Hb', 'Plt', 'wbc', 'Na'
        , 'Urea', 'Alb', 'Hct', 'K', 'Lactate'))
order by person_id, measurement_datetime

In [ ]:
# merge lab values
cols_to_use = lab.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, lab[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
del lab

In [ ]:
# @title Vasopressors :-( unable to combine by range in pandas
%%bigquery vp --project $PROJECT_ID
with vlist as (
  select distinct person_id
from measurement m inner join concept c on m.measurement_concept_id = c.concept_id
where concept_name like '%ventila%'
),
vp as (
select person_id, drug_exposure_start_datetime, drug_exposure_end_datetime
  , case when drug_concept_id in (19006871) then 'argipressin'
       when drug_concept_id in (1321341 ) then 'norepinephrine'
       when drug_concept_id in (1337720) then 'dobutamine'
       when drug_concept_id in (1337860) then 'dopamine'
       end as concept_name
, quantity
from drug_exposure
where drug_concept_id in (19006871 -- argipressin
    , 1321341 -- norepinephrine
    , 1337720 -- dobutamine
    , 1337860 -- dopamine
)
  and person_id in (select person_id from vlist)
)
select * from vp
PIVOT (max(quantity) for concept_name in ('argipressin', 'norepinephrine', 'dobutamine', 'dopamine'))
order by person_id, drug_exposure_start_datetime, drug_exposure_end_datetime

In [ ]:
# merge vp values
# unable to merge on date range

#cols_to_use = vp.columns.difference(ventpat.columns)
#cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
#ventpat = pd.merge(ventpat, vp[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
#                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
# unable to merge properly.
del vp

In [ ]:
# @title I/O
%%bigquery fluid --project $PROJECT_ID
WITH vlist AS (
  SELECT DISTINCT person_id
  FROM measurement m
  JOIN concept c ON m.measurement_concept_id = c.concept_id
  WHERE concept_name LIKE '%ventila%' -- we only want vent patientss
),
labeled_data AS (
  SELECT
    m.person_id,
    m.measurement_datetime,
    m.value_as_number,
    -- Classify measurements based on configured IDs
    CASE
      WHEN m.measurement_concept_id IN (
        3037253, -- Intravascular
        3010494, -- Enteral
        3006552  -- Oral
      ) THEN 'Input'
      WHEN m.measurement_concept_id IN (
        3014315, -- Urine
        3011087, -- Stool
        3026556, -- Chest Tube
        3018767, -- Ventricle Drain
        21491183, -- GI Drain
        3020433  -- Misc
      ) THEN 'Output'
      ELSE NULL
    END as category
  FROM measurement m
  WHERE m.measurement_concept_id IN (
    3037253, -- Intravascular
        3010494, -- Enteral
        3006552,  -- Oral
    3014315, -- Urine
        3011087, -- Stool
        3026556, -- Chest Tube
        3018767, -- Ventricle Drain
        21491183, -- GI Drain
        3020433  -- Misc
  )
  AND m.person_id IN (SELECT person_id FROM vlist)
)
SELECT * from labeled_data
PIVOT (
  MAX(value_as_number) FOR category IN ('Input', 'Output')
)
ORDER BY person_id, measurement_datetime


In [ ]:
# merge fluid values
cols_to_use = fluid.columns.difference(ventpat.columns)
cols_to_use = cols_to_use.insert(0, ['person_id', 'measurement_datetime'])
ventpat = pd.merge(ventpat, fluid[cols_to_use], how='left', left_on=['person_id', 'measurement_datetime']
                          , right_on=['person_id', 'measurement_datetime'])

In [ ]:
del fluid

# START HERE!

In [ ]:
df = ventpat

In [ ]:
del ventpat

In [ ]:
df['seconds_to_wean'] = (df['measurement_datetime'] - df['wean_start']).dt.total_seconds()

In [ ]:
# df[df['Output']>0]
df[df['person_id']==1470] # random case check

In [ ]:
# @title check missing values?
# what should i do with the empty values?
missing_values_count = df.isna().sum()
sorted_missing_values = missing_values_count.sort_values(ascending=False)
sorted_missing_values


In [ ]:

df.describe()
